In [1]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F


class SkipGramModel(nn.Module):

    def __init__(self, emb_size_u, emb_size_v,emb_dimension):
        super(SkipGramModel, self).__init__()
        self.emb_dimension = emb_dimension
        self.u_embeddings = nn.Embedding(emb_size_u, emb_dimension, sparse=False)
        self.v_embeddings = nn.Embedding(emb_size_v, emb_dimension, sparse=False)
        self.init_emb()
        

    def init_emb(self):
        initrange = 0.5 / self.emb_dimension
        self.u_embeddings.weight.data.uniform_(-initrange, initrange)
        self.v_embeddings.weight.data.uniform_(-0, 0)
        
    def forward(self, pos_u, pos_v,neg_v):
        emb_u = self.u_embeddings(pos_u)
        neg_v = neg_v.view(len(pos_u),-1)
        samples = torch.cat([pos_v,Variable(neg_v)],1)
        emb_v = self.v_embeddings(samples)
        score = torch.bmm(emb_v, emb_u.unsqueeze(2)).squeeze()
        score[:,1:]=score[:,1:].neg()
        score = F.logsigmoid(score)
        return -1 * (torch.sum(score))/ pos_u.size(0)

In [5]:
from torch.utils.data import Dataset
from collections import defaultdict
from gensim.models.doc2vec import TaggedDocument
import random

class wDataSet(Dataset):
    def __init__(self, dataset, power=0.75, neg_samples=10):
        self.pairs = self.generate_pairs(dataset,5)
        self.neg_samples=neg_samples
        
    def generate_pairs(self, dataset, ctx_window):
            pairs = []
            for sentence in dataset:
                for i,word in enumerate(sentence):
                    for j in range(1,ctx_window):
                        if(i+j<len(sentence[i])):
                            pairs.append((word,sentence[i+j]))
                        if((i-j)>0):
                            pairs.append((word,sentence[i-j]))
                            
            return pairs
                            
        
    def __len__(self):
        return len(pairs)
        
    def __getitem__(self, idx):
        return pairs[idx]



In [3]:
from gensim.test.utils import datapath
import gensim.downloader as api
#sentences = LineSentence(datapath('lee_background.cor'))
dataset = api.load('text8')
print(type(dataset))
p = []
for x in dataset: 
    p.append(x)
    
"""
print(len(p))
sum = 0
for x in p: 
    sum += len(x)
print(sum/len(p))
print(len(p[0]))
"""

<class 'text8.Dataset'>


'\nprint(len(p))\nsum = 0\nfor x in p: \n    sum += len(x)\nprint(sum/len(p))\nprint(len(p[0]))\n'

In [61]:
#dataset = wDataSet(p)

AttributeError: 'wDataSet' object has no attribute 'generate_pairs'

In [4]:
from collections import Counter
tmp=[['one','tw0'],['tree','four']]
print(type(tmp))
print(type(p))
counts = Counter(p[0])
vocab = sorted(counts, key=counts.get, reverse=True)
vocab_to_int = {word: ii for ii, word in enumerate(vocab, 0)}
print(counts)
print(vocab)
print(vocab_to_int)

<class 'list'>
<class 'list'>
Counter({'the': 521, 'of': 357, 'and': 302, 'in': 250, 'to': 227, 'a': 184, 'as': 133, 'one': 130, 'is': 110, 'anarchism': 102, 'that': 102, 'with': 85, 'or': 77, 'anarchists': 76, 'anarchist': 69, 's': 69, 'nine': 64, 'for': 63, 'by': 57, 'zero': 57, 'be': 55, 'autism': 55, 'they': 49, 'are': 45, 'an': 44, 'some': 44, 'have': 44, 'it': 43, 'not': 43, 'people': 43, 'was': 42, 'autistic': 42, 'such': 39, 'this': 38, 'many': 38, 'eight': 38, 'from': 37, 'on': 36, 'social': 34, 'their': 34, 'other': 33, 'two': 33, 'anarcho': 33, 'also': 32, 'often': 30, 'see': 29, 'most': 28, 'movement': 27, 'can': 27, 'at': 26, 'which': 26, 'may': 26, 'first': 25, 'state': 25, 'three': 24, 'what': 23, 'his': 23, 'its': 22, 'he': 22, 'society': 21, 'has': 21, 'more': 21, 'six': 20, 'children': 20, 'movements': 19, 'use': 19, 'property': 19, 'ideas': 18, 'language': 18, 'post': 18, 'them': 18, 'but': 17, 'anti': 17, 'war': 17, 'seven': 17, 'groups': 17, 'term': 16, 'against': 

In [ ]:
from collections import Counter
tmp=[['one','tw0'],['tree','four']]
print(type(tmp))
print(type(p))
small_dataset=p[0]
vocabulary = []
for word in small_dataset:
        if word not in vocabulary:
            vocabulary.append(word)
#print(vocab)
word2idx = {w: idx for (idx, w) in enumerate(vocabulary)}
idx2word = {idx: w for (idx, w) in enumerate(vocabulary)}

v_embeddings = nn.Embedding(5, 1, sparse=False)
u_embeddings = nn.Embedding(5, 1, sparse=False)
initrange = 0.5 / 5
u_embeddings.weight.data.uniform_(-initrange, initrange)
v_embeddings.weight.data.uniform_(-0, 0)
print(u_embeddings)



# Then turn the word index into actual word vector
vocab = {"some": 0, "words": 1}
word_indexes = [vocab[w] for w in ["some", "words"]] 
print(word_indexes)
word_vectors = u_embeddings(one_hot_vector(2))
print(word_vectors)